In [1]:
import pandas as pd 
import numpy as np 

estado dos carros:
cp(j), cd(j), cb(j), cp(j), ct(j)

M=10 pisos, N=2 carros

estados dos pisos:

    ff(i)[], fb(i), ftc(i)
    for ts=1:Ts    % Ts=3600 para 1 hora
        for ts=1:Ts
            for i=1:M (para cada andar)
                if ftc(i) <= ts then
                    ff(i) <-- [ftc, ta]
                    ftc(i) = ftc(i) + exp(1/lambda);
                    atribui_carro();
                    c = atribui_car()
                    cb(c) = 1;
                    d = gera_dest();
                end if
            endfor-ftc
            for j=1:N (para cada elevador)
                if cp(j) mod 30 <> 0 then
                    if cd(j) = 1 then
                        cp(j) = cp(j) + 1;
                    if cd(j) = -1 then
                        cp(j) = cp(j) -1;
                    else ERRO1;

                if cp(j) mod 30 = 0 then
                    f = quoc(cp(j) mod 30);
                    case cd(j) do
                        1: fazer()
                        -1: fazer()
                        0: ocioso();
    end-for-ts
Lembrete: toda vez que retira passageiros da s filas, salva os dados daquele passageiro [ftc, ta] num arquivo, que poder ser em disco ou em memória, para calcular média e variância do tempo de espera.

In [2]:
#carro
#variaveis do carro:  (CPj, CDj, CBj, CGj, CTj) car position, 
#car direction, clicked button, car get passengers waiting, temporizer
class Car:
    def __init__(self, name, car_position = 0, passengers = [], car_direction = 0, pressed_buttons = [], car_passengers_waiting = set(), temporizer = 0):
        self.car_position = car_position
        self.car_direction = car_direction
        self.pressed_buttons = pressed_buttons #destinos dos passageiros
        self.car_passengers_waiting = car_passengers_waiting #andares que passageiros entrarão
        self.temporizer = temporizer
        self.name = name
        self.passengers = passengers
        
    def car_action(self, floor):
        if (floor.number in self.pressed_buttons):
            #retira os  passageiros
            for p in self.passengers:
                if (p.destination == floor.number):
                    print("passageiro sai")
                    if (floor.number in self.pressed_buttons):
                            self.pressed_buttons.remove(floor.number)
                    #atualiza o attendance
                    p.attendence = self.temporizer
                    #salva nos passageiros atendidos
                    passengers_served.append(p)
                    #remove do elevador
                    self.passengers.remove(p)
                
        
        print("{} in {}".format(floor.number, self.car_passengers_waiting))
        if (floor.number in self.car_passengers_waiting):
            print("Andar {}".format(floor.number))
            #pega os  passageiros
            for p in floor.passengers_queue:
                if (p.selected_car == self.name):
                    if (p.destination > floor.number and self.car_direction == 1):
                        #entra
                        if (floor.number in self.car_passengers_waiting):
                            self.car_passengers_waiting.remove(floor.number)
                        #adiciona no elevador
                        self.passengers.append(p)
                        #remove do andar
                        floor.passengers_queue.remove(p)                        
                        print("passageiro entra")
                        self.pressed_buttons.append(p.destination)
                        p.boarding = self.temporizer
                        
                    elif (p.destination < floor.number and self.car_direction == -1):
                        #entra
                        if (floor.number in self.car_passengers_waiting):
                            self.car_passengers_waiting.remove(floor.number)
                        #remove do andar
                        #adiciona no elevador
                        print("passageiro entra")
                        self.pressed_buttons.append(p.destination)
                        p.boarding = self.temporizer
                    else:
                        #print("ERROR2")
                        #print("destination {} - andar {} - direcao {}".format(p.destination, floor.number, self.car_direction))
                        pass
                        
        # atualiza direcao
        if (self.car_direction == 0):
            if (len(self.pressed_buttons) == 0):
                if (len(self.car_passengers_waiting) == 0):
                    print("car {} no action".format(self.name))
                else:
                    solicited_floor = list(self.car_passengers_waiting)[0]
                    current_floor = int(self.car_position/30)
                    if(solicited_floor > current_floor):
                        self.car_direction = 1
                        print("car {} - going up".format(self.name))
                    elif(solicited_floor < current_floor):
                        self.car_direction = -1
                        print("car {} - going down".format(self.name))
            else:
                solicited_floor = self.pressed_button[0]
                current_floor = int(self.car_position/30)
                if(solicited_floor > current_floor):
                    self.car_direction = 1
                    print("car {} - going up".format(self.name))
                elif(solicited_floor < current_floor):
                    self.car_direction = -1
                    print("car {} - going down".format(self.name))
                
        elif(self.car_direction == 1):
            if (len(self.pressed_buttons) == 0):
                if (len(self.car_passengers_waiting) == 0):
                    #se nao tem requisicoes para
                    self.car_direction = 0
                    print("car {} no action".format(self.name))
                else:
                    current_floor = int(self.car_position/30)
                    keep_going = False
                    #verifica se tem requisicao pra subir
                    for solicited_floors in self.car_passengers_waiting:
                        if(solicited_floors > current_floor):
                            print("car {} - going up".format(self.name))
                            keep_going = True
                            break
                            
                    #verifica se tem pra descer
                    if(not keep_going):
                        for solicited_floors in self.car_passengers_waiting:
                            if(solicited_floors < current_floor):
                                self.car_direction = -1
                                print("car {} - going down".format(self.name))
                                break
                                
            else:
                current_floor = int(self.car_position/30)
                keep_going = False
                for solicited_floors in self.pressed_buttons:
                    if(solicited_floors > current_floor):
                        self.car_direction = 1
                        print("car {} - going up".format(self.name))
                        keep_going = True
                        break
                        
                for solicited_floors in self.car_passengers_waiting:
                    if(solicited_floors > current_floor):
                        self.car_direction = 1
                        print("car {} - going up".format(self.name))
                        keep_going = True
                        break   
                        
                if(not keep_going):
                    for solicited_floors in self.pressed_buttons:
                        if(solicited_floors < current_floor):
                            self.car_direction = -1
                            print("car {} - going down".format(self.name))
                            break
                        
                    for solicited_floors in self.car_passengers_waiting:
                        if(solicited_floors < current_floor):
                            self.car_direction = -1
                            print("car {} - going down".format(self.name))
                            break
                            
        else:
            if (len(self.pressed_buttons) == 0):
                if (len(self.car_passengers_waiting) == 0):
                    #se nao tem requisicoes para
                    self.car_direction = 0
                    print("car {} no action".format(self.name))
                else:
                    current_floor = int(self.car_position/30)
                    keep_going = False
                    #verifica se tem requisicao pra descer
                    for solicited_floors in self.car_passengers_waiting:
                        if(solicited_floors < current_floor):
                            print("car {} - going down".format(self.name))
                            keep_going = True
                            break
                            
                    #verifica se tem pra subir
                    if(not keep_going):
                        for solicited_floors in self.car_passengers_waiting:
                            if(solicited_floors > current_floor):
                                self.car_direction = 1
                                print("car {} - going up".format(self.name))
                                break
            else:
                current_floor = int(self.car_position/30)
                keep_going = False
                for solicited_floors in self.pressed_buttons:
                    if(solicited_floors < current_floor):
                        print("car {} - going down".format(self.name))
                        keep_going = True
                        break
                        
                for solicited_floors in self.car_passengers_waiting:
                    if(solicited_floors < current_floor):
                        print("car {} - going down".format(self.name))
                        keep_going = True
                        break   
                        
                if(not keep_going):
                    for solicited_floors in self.pressed_buttons:
                        if(solicited_floors > current_floor):
                            self.car_direction = 1
                            print("car {} - going up".format(self.name))
                            break
                        
                    for solicited_floors in self.car_passengers_waiting:
                        if(solicited_floors > current_floor):
                            self.car_direction = 1
                            print("car {} - going up".format(self.name))
                            break
        
            
    def new_passager_waiting(self, floor):
        self.car_passengers_waiting.add(floor.number)
        

In [3]:
from random import choice
#andar
#ff(i)[], fb(i), ftc(i) ->
#fila de pessoas, botao descer, botao subir, contador proxima chegada de pessoa
class Floor:
    def __init__(self, number, passengers_queue = [], up_button = 0, down_button =  0, passagers_next_flow = int(np.random.exponential(1/0.004))):
        self.passengers_queue = passengers_queue
        self.up_button = up_button
        self.down_button = down_button
        self.passagers_next_flow = passagers_next_flow
        self.number = number
        
    def random_destination(self):
        return choice([i for i in range(0,9) if i not in [self.number]])
        

In [4]:
class Passager:
    def __init__(self, destination, arrive, selected_car, boarding = 1000000, attendance = 1000000):
        self.destination = destination
        self.arrive = arrive
        self.boarding = boarding
        self.attendance = attendance
        self.selected_car = selected_car

In [5]:
#inicializacao 3600 iteracoes, 2 carros e 10 andares (0 a 9)
timer = 3600
floor_number = 10
car_number = 2
lambda_value = 0.005

passengers_served = []
cars = []
floors = []
names = ['A', 'B', 'C']
for x in range(car_number):
    cars.append(Car(name = names[x], pressed_buttons = [], passengers = [] , car_passengers_waiting = set()))

for x in range(floor_number):
    #print(int(np.random.exponential(1/0.004)))
    floors.append(Floor(number = x, passagers_next_flow = int(np.random.exponential(1/0.004))))

In [6]:
def select_best_car():
    #substituir carro mais proximo    
    return cars[0]

In [7]:
for iteration in range(timer):
    floor = None 
    car = None
    selected_car = None
    for floor in floors:
        if floor.passagers_next_flow == iteration:
            #print("Andar {} - tempo {} - flow do andar {}".format(floor.number, iteration, floor.passagers_next_flow))
            e = np.random.exponential(1/lambda_value)
            floor.passagers_next_flow += e
            d = floor.random_destination()
            selected_car = select_best_car()
            print("Floor: {} - Best Car {}".format(floor.number, selected_car.name))
            p = Passager(arrive = iteration, destination = d, selected_car = selected_car.name)
            floor.passengers_queue.append(p)
            selected_car.new_passager_waiting(floor)
    
    for car in cars:
        car.temporizer = iteration
        print("Carro: {} - Andar {} - fila de passageiros {} - fila de andares {}".format(car.name, int(car.car_position/30), car.pressed_buttons, car.car_passengers_waiting))
        if (car.car_position % 30 != 0):
            if (car.car_direction == 1):
                car.car_position += 1
            elif (car.car_direction == -1):
                car.car_position -= 1
            else:
                print("ERROR1")
            
        elif (car.car_position % 30 == 0):
            f = int(car.car_position/30)
            floor = floors[f]
            car.car_action(floor)
            if (car.car_direction == 1):
                car.car_position += 1
            elif (car.car_direction == -1):
                car.car_position -= 1
            elif (len(car.pressed_buttons) == 0 and len(car.car_passengers_waiting) == 0):
                pass
            else:
                print("ERROR1")
            

Carro: A - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car A no action
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car A no action
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car A no action
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car A no action
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car A no action
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Floor: 2 - Best Car A
Carro: A - Andar 0 - fila de

0 in set()
car B no action
Carro: A - Andar 7 - fila de passageiros [1, 2, 1, 2, 1, 2, 4] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 7 - fila de passageiros [1, 2, 1, 2, 1, 2, 4] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 7 - fila de passageiros [1, 2, 1, 2, 1, 2, 4] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 7 - fila de passageiros [1, 2, 1, 2, 1, 2, 4] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 7 - fila de passageiros [1, 2, 1, 2, 1, 2, 4] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 7 - fila de passageiros [1, 2, 1, 2, 1, 2

car B no action
Carro: A - Andar 4 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 4 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
4 in {0, 9}
car A - going down
car A - going down
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 3 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 3 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 3 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de 

0 in set()
car B no action
Carro: A - Andar 6 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 6 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 6 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 6 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 6 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no actio

Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 1 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 1 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 1 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
1 in {0, 9}
car A - going down
car A - going down
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 0 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] - fila de andares {0, 9}
Carro: B - Andar 0 - fila de passageiros [] - fila de andares set()
0 in set()
car B no action
Carro: A - Andar 0 - fila de passageiros [1, 2, 1, 2, 1, 2, 4, 1, 2, 5, 0] 